In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
from language_3d_io import GeoWordsDataSet
from general_tools.notebook.tf import reset_tf_graph

In [3]:
import tensorflow as tf
import numpy as np
from general_tools.in_out.basics import unpickle_data
from tflearn.layers.core import fully_connected
from tf_lab.point_clouds.encoders_decoders import decoder_with_fc_only

In [49]:
from tf_lab.rnn import deep_lstm, length_of_sequence, last_relevant_rnn_output, get_state_variables,\
                       get_state_update_op, get_state_reset_op

In [50]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
n_hidden = 384
n_layers = 1
frame_size = 100
n_classes = 3
batch_size = 40
max_steps = 30
geo_feat_size = 128 * 3

In [52]:

training_data = unpickle_data('train_100_close.pkl').next()
test_data = unpickle_data('test_40_close.pkl').next()

In [53]:
reset_tf_graph()

In [54]:
input_sequence = tf.placeholder(tf.float32, [None, max_steps, frame_size])

input_geometry = tf.placeholder(tf.float32, [None, geo_feat_size])

target = tf.placeholder(tf.float32, [None, n_classes])

dropout_prob = tf.placeholder(tf.float32)

deep_lstm_model = deep_lstm(n_layers, n_hidden, dropout_prob)

len_seq = length_of_sequence(input_sequence)

states = get_state_variables(batch_size, deep_lstm_model)

# Unroll the LSTM

# # tf.nn.dynamic_rnn returns the output activations and last hidden state.
# # The output will be of size batch_size x max_step x n_hidden, 
# # but with the last being zero vectors for sequences shorter than the maximum length.
# # The final_state contains the last state (batch_size x n_hidden) of each layer of the stacked-deep-rnn.
rnn_outputs, last_states = tf.nn.dynamic_rnn(deep_lstm_model, input_sequence, dtype=tf.float32, sequence_length=len_seq,\
                                            initial_state=states)

reset_states_with_zero = get_state_reset_op(states, deep_lstm_model, batch_size)
reset_h_with_geo = states[0][1].assign(input_geometry)

rnn_out = last_relevant_rnn_output(rnn_outputs, len_seq)
input_to_decoder = rnn_out 
# input_to_decoder = tf.concat(1, [rnn_out, input_geometry])
# input_to_decoder = tf.concat(1, [joint_last_state, input_geometry])

In [58]:
logits = decoder_with_fc_only(input_to_decoder, layer_sizes=[100, n_classes])

prediction = tf.argmax(logits, axis=1)

target_ = tf.argmax(target, axis=1)

correct_pred = tf.equal(prediction, target_)

avg_accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [60]:
def cost(output, target):
    # Compute cross entropy for each frame.
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(output, target)
    return tf.reduce_mean(cross_entropy)

def optimizer_step(loss, learning_rate=0.003):
    opt = tf.train.AdamOptimizer(learning_rate)
    grad_params = opt.compute_gradients(loss)
    capped_gp = [(tf.clip_by_value(grad, -5., 5.), param) for grad, param in grad_params]
    return opt.apply_gradients(capped_gp)

In [61]:
problem_loss = cost(logits, target)
problem_opt_step = optimizer_step(problem_loss)

In [62]:
gpu_config = tf.ConfigProto()
gpu_config.gpu_options.allow_growth = True
init = tf.global_variables_initializer()

In [66]:
dropout_pr = 0.5
display_step = 1
stats = []

batches_for_epoch = training_data.num_examples / batch_size

sess = tf.Session(config=gpu_config)
sess.run(init)
sess.run(reset_states_with_zero)

for epoch in range(30):
    epoch_loss = 0
    epoch_perf = 0    

    for _ in range(batches_for_epoch):    
        words_i, geo_i, labels_i = training_data.next_batch(batch_size)
        
        feed_dict = {input_sequence: words_i,
                     target: labels_i,
                     input_geometry: geo_i,
                     dropout_prob: dropout_pr}
        
        sess.run([reset_h_with_geo], feed_dict=feed_dict)
        _, step_loss, step_perf = sess.run([problem_opt_step, problem_loss, avg_accuracy], feed_dict=feed_dict)
        
        epoch_loss += step_loss
        epoch_perf += step_perf
    
    epoch_loss /= batches_for_epoch
    epoch_perf /= batches_for_epoch
    if epoch % display_step == 0 :
        stats.append((epoch_loss, epoch_perf))
        print epoch + 1, stats[-1]

1 (1.1874086161454518, 0.33333334202567738)
2 (1.1097140808900197, 0.3395833472410838)
3 (1.1027159988880157, 0.38958333432674408)
4 (1.069871614376704, 0.43125000596046448)
5 (1.0402942001819611, 0.45625000695387524)
6 (0.98660088578859961, 0.48958333084980649)
7 (0.95301790535449982, 0.51458333929379785)
8 (0.89520235359668732, 0.55833335469166434)
9 (0.78936024010181427, 0.62083334724108374)
10 (0.73291848599910736, 0.6583333512147268)
11 (0.71659406522909797, 0.69375000894069672)
12 (0.53863394260406494, 0.7520833512147268)
13 (0.46037585039933521, 0.81250002483526862)
14 (0.42878260587652522, 0.83333336313565576)
15 (0.37416063994169235, 0.85416668156782782)
16 (0.38217221821347874, 0.83333335320154822)
17 (0.35676431655883789, 0.84375003476937616)
18 (0.31641138717532158, 0.86458335816860199)
19 (0.27267804990212124, 0.90000002086162567)
20 (0.19544699477652708, 0.91875000298023224)
21 (0.12822379606465498, 0.95208333432674408)
22 (0.11543917004019022, 0.9583333283662796)
23 (0.1

In [69]:
def try_model_with_test_data(test_data, batch_size):

    batches_for_epoch = test_data.num_examples / batch_size
    sess.run(reset_states_with_zero)

    loss = 0
    perf = 0
    
    for _ in range(batches_for_epoch):
        words_i, geo_i, labels_i = test_data.next_batch(batch_size)
        
        feed_dict = {input_sequence: words_i,
                     target: labels_i,
                     input_geometry: geo_i,
                     dropout_prob: 0.0}
        
        sess.run([reset_h_with_geo], feed_dict=feed_dict)
        step_loss, step_perf = sess.run([problem_loss, avg_accuracy], feed_dict=feed_dict)        
        loss += step_loss
        perf += step_perf
    return loss, perf

try_model_with_test_data(test_data, batch_size)

(2.5297815799713135, 0.47499999403953552)

In [105]:
# Load pre-trained word-embedding.
vocab_size = int(4e5)
embedding_dim = 100

word_embedding = tf.Variable(tf.constant(0.0, shape=[vocab_size, embedding_dim]), trainable=False, name="word_embedding")
embedding_placeholder = tf.placeholder(tf.float32, [vocab_size, embedding_dim])
embedding_init = word_embedding.assign(embedding_placeholder)

sess = tf.Session()
sess.run(embedding_init, feed_dict={embedding_placeholder: lala})

def embedding_dictionary_to_matrix(in_dict):
    return np.array(in_dict.values())

array([[-0.049107  ,  1.08360004, -0.96898001, ..., -0.35431999,
         0.46728   ,  0.34469   ],
       [-0.59596997, -0.022     ,  0.55533999, ...,  0.042286  ,
        -0.3973    ,  0.42844   ],
       [ 0.21253   , -0.094895  ,  0.53437001, ...,  0.29701   ,
        -0.12437   ,  0.20121001],
       ..., 
       [ 1.0632    , -0.028459  , -0.46533999, ...,  0.039003  ,
        -1.03390002, -0.52419001],
       [ 0.31733   , -0.20597   ,  0.13283999, ...,  0.65559   ,
        -0.70892   ,  0.13138001],
       [-1.00580001, -0.53103   , -0.15195   , ...,  0.20653   ,
         0.26804   , -0.95627999]], dtype=float32)

In [ ]:
def last_hidden_states(output):
    hidden_states = []
    for i in range(len(output)):
        hidden_states.append(final_state[i].h)

    if len(hidden_states) > 1:
        joint_last_state = tf.concat_v2(hidden_states, axis=1)
    else:
        joint_last_state = hidden_states[0]
    return hidden_states
